In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import Normalizer, StandardScaler
import random

import time

In [23]:
#!pyspark --version
#!spark-submit --version
#!spark-shell --version
#!spark-sql --version

In [24]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 pyspark-shell'

In [25]:
kafka_topic_name = "Topic"
kafka_bootstrap_servers = 'sandbox.hortonworks.com:6667'

In [26]:
spark = SparkSession \
        .builder \
        .appName("Structured Streaming ") \
        .master("local[*]") \
        .getOrCreate()

In [27]:
spark.sparkContext.setLogLevel("ERROR")

In [28]:
# Construct a streaming DataFrame that reads from topic
flower_df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", 'sandbox.hortonworks.com:6667') \
        .option("subscribe", kafka_topic_name) \
        .option("startingOffsets", "latest") \
        .load()

In [29]:
flower_df1 = flower_df.selectExpr("CAST(value AS STRING)", "timestamp")


#flower_schema_string = "order_id INT,sepal_length DOUBLE,sepal_length DOUBLE,sepal_length DOUBLE,sepal_length DOUBLE,species STRING"
flower_schema_string = "order_id INT,sepal_length DOUBLE,sepal_width DOUBLE,petal_length DOUBLE,petal_width DOUBLE,variety STRING"



flower_df2 = flower_df1 \
        .select(from_csv(col("value"), flower_schema_string) \
                .alias("flower"), "timestamp")


flower_df3 = flower_df2.select("flower.*", "timestamp")

    
flower_df3.createOrReplaceTempView("flower_find");
song_find_text = spark.sql("SELECT * FROM flower_find")
flower_agg_write_stream = song_find_text \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format("memory") \
        .queryName("testedTable") \
        .start()

flower_agg_write_stream.awaitTermination(1)

False

In [30]:
flower_agg_write_stream.status

{'message': 'Getting offsets from KafkaV2[Subscribe[Topic]]',
 'isDataAvailable': False,
 'isTriggerActive': True}

In [32]:
spark.sql("select * from testedTable").show()

+--------+------------+-----------+------------+-----------+---------+--------------------+
|order_id|sepal_length|sepal_width|petal_length|petal_width|  variety|           timestamp|
+--------+------------+-----------+------------+-----------+---------+--------------------+
|     113|         5.7|        2.5|         5.0|        2.0|Virginica|2022-06-12 17:17:...|
|     114|         5.8|        2.8|         5.1|        2.4|Virginica|2022-06-12 17:17:...|
|     115|         6.4|        3.2|         5.3|        2.3|Virginica|2022-06-12 17:17:...|
|     116|         6.5|        3.0|         5.5|        1.8|Virginica|2022-06-12 17:17:...|
|     117|         7.7|        3.8|         6.7|        2.2|Virginica|2022-06-12 17:17:...|
|     118|         7.7|        2.6|         6.9|        2.3|Virginica|2022-06-12 17:17:...|
|     119|         6.0|        2.2|         5.0|        1.5|Virginica|2022-06-12 17:17:...|
|     120|         6.9|        3.2|         5.7|        2.3|Virginica|2022-06-12

In [21]:
spark.stop()

22/06/12 17:16:35 ERROR MicroBatchExecution: Query testedTable [id = 1102cfc0-8c8a-4918-80d1-b82e856859c9, runId = f7f44405-ccdb-405c-a015-706ee4e5ae55] terminated with error
java.lang.NullPointerException
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.getSortedExecutorList(KafkaOffsetReaderConsumer.scala:484)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.getOffsetRangesFromResolvedOffsets(KafkaOffsetReaderConsumer.scala:539)
	at org.apache.spark.sql.kafka010.KafkaMicroBatchStream.planInputPartitions(KafkaMicroBatchStream.scala:183)
	at org.apache.spark.sql.execution.datasources.v2.MicroBatchScanExec.partitions$lzycompute(MicroBatchScanExec.scala:44)
	at org.apache.spark.sql.execution.datasources.v2.MicroBatchScanExec.partitions(MicroBatchScanExec.scala:44)
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2ScanExecBase.supportsColumnar(DataSourceV2ScanExecBase.scala:93)
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2ScanExecBase.sup